MACHINE FAILURE PREDICTION

In [7]:
import numpy as np
import pandas as pd 

In [8]:
machine = pd.read_csv("ongc_data.csv")
machine.head()

,Machine ID,Location,Machine Age,Operating Hours,Temperature,Pressure,Vibration,Last Maintenance Date,Failure
0,M_1,Offshore Platform 1,7,8905,52.710458,207187.772428,3.424334,2021-08-11,1
1,M_2,Offshore Platform 1,20,3858,52.561077,409338.911282,9.954492,2021-08-31,0
2,M_3,Refinery 1,15,7095,52.768825,475921.873983,0.656818,2019-09-18,0
3,M_4,Offshore Platform 2,11,3219,136.632773,174270.563639,4.705372,2019-09-22,0
4,M_5,Offshore Platform 2,8,8058,119.275650,308153.881075,8.312978,2023-03-27,0


In [12]:
machine['Location'].value_counts()


Location
Refinery 2             20227
Offshore Platform 1    20099
Offshore Platform 2    19963
Pipeline 1             19930
Refinery 1             19781
Name: count, dtype: int64

ASK SIX QUESTIONS


In [13]:
machine.shape

(100000, 9)

In [14]:
machine.isnull().sum()

Machine ID               0
Location                 0
Machine Age              0
Operating Hours          0
Temperature              0
Pressure                 0
Vibration                0
Last Maintenance Date    0
Failure                  0
dtype: int64

In [15]:
machine.duplicated().sum()

np.int64(0)

In [16]:
machine.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 9 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Machine ID             100000 non-null  object 
 1   Location               100000 non-null  object 
 2   Machine Age            100000 non-null  int64  
 3   Operating Hours        100000 non-null  int64  
 4   Temperature            100000 non-null  float64
 5   Pressure               100000 non-null  float64
 6   Vibration              100000 non-null  float64
 7   Last Maintenance Date  100000 non-null  object 
 8   Failure                100000 non-null  int64  
dtypes: float64(3), int64(3), object(3)
memory usage: 6.9+ MB


In [21]:
import pandas as pd

# Read the CSV file into a DataFrame
machine = pd.read_csv("ongc_data.csv")

# Check data types of columns
print(machine.dtypes)

# Select only numeric columns
numeric_columns = machine.select_dtypes(include=['float64', 'int64']).columns
machine_numeric = machine[numeric_columns]

# Compute correlations
correlations = machine_numeric.corr()



Machine ID                object
Location                  object
Machine Age                int64
Operating Hours            int64
Temperature              float64
Pressure                 float64
Vibration                float64
Last Maintenance Date     object
Failure                    int64
dtype: object


ENCODING THE TARGET VALUE

In [25]:

#from sklearn.preprocessing import LabelEncoder
#le = LabelEncoder()

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
import joblib

# Load data from the correct CSV file
data = pd.read_csv('ongc_data.csv')

# Convert 'Last Maintenance Date' to datetime
data['Last Maintenance Date'] = pd.to_datetime(data['Last Maintenance Date'])

# Separate features and target
X = data.drop(['Machine ID', 'Failure'], axis=1)
y = data['Failure']

# Preprocessing pipeline
numeric_features = ['Machine Age', 'Operating Hours', 'Temperature', 'Pressure', 'Vibration']
categorical_features = ['Location']

numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))  # Handle unknown categories
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Create the preprocessing and training pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', RandomForestClassifier(random_state=42))])

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Training
pipeline.fit(X_train, y_train)

# Save the model and pipeline
joblib.dump(pipeline, 'machine_failure_model.pkl')



['machine_failure_model.pkl']

Prediction code


In [ ]:
import joblib
import pandas as pd
from datetime import datetime

# Load the trained model
pipeline = joblib.load('machine_failure_model.pkl')

def get_user_input():
    # Collect user input
    location = input("Enter the location: ")
    machine_age = int(input("Enter the machine age: "))
    operating_hours = int(input("Enter the operating hours: "))
    temperature = float(input("Enter the temperature: "))
    pressure = float(input("Enter the pressure: "))
    vibration = float(input("Enter the vibration: "))
    last_maintenance_date = input("Enter the last maintenance date (YYYY-MM-DD): ")

    # Convert date to datetime object
    last_maintenance_date = datetime.strptime(last_maintenance_date, '%Y-%m-%d')

    # Create a DataFrame for the input data
    user_data = pd.DataFrame({
        'Location': [location],
        'Machine Age': [machine_age],
        'Operating Hours': [operating_hours],
        'Temperature': [temperature],
        'Pressure': [pressure],
        'Vibration': [vibration],
        'Last Maintenance Date': [last_maintenance_date]
    })

    return user_data

def predict_failure(user_data):
    # Predict using the trained model
    prediction = pipeline.predict(user_data)
    probability = pipeline.predict_proba(user_data)

    return prediction, probability

if __name__ == "__main__":
    user_data = get_user_input()
    prediction, probability = predict_failure(user_data)

    print("Prediction (1: Failure, 0: No Failure):", prediction[0])
    print("Probability of Failure:", probability[0][1])


Connecting backend with front end 

In [9]:
import pickle
pickle.dump(train_test_split,open('model.pkl','wb'))